In [47]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt
import os
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn import ensemble, tree, linear_model
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
from scipy.stats import pearsonr

plt.style.use('seaborn')
import re
from scipy import stats
from scipy.stats import chisquare
from scipy.stats import chi2_contingency
from scipy.stats import f_oneway
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.stats import ks_2samp
from scipy.stats import norm
import missingno as msno
import datetime
import pickle

In [48]:
# load dataframes
with open("../pickles/df_after_outliers_missing.pkl", "rb") as f:
    df = pickle.load(f)

with open("../pickles/df_appearances.pkl", "rb") as f:
    df_appearances = pickle.load(f)

with open("../pickles/df_games.pkl", "rb") as f:
    df_games = pickle.load(f)

with open("../pickles/df_shots.pkl", "rb") as f:
    df_shots = pickle.load(f)

with open("../pickles/player_game_team_mapping.pkl", "rb") as f:
    df_player_game_team_mapping = pickle.load(f)
    


In [49]:
display(df)
display(df_appearances)
display(df_games)
display(df_shots)
display(df_player_game_team_mapping)


,gameID,leagueID,season,date,homeTeamID,awayTeamID,home_Goals,away_Goals,home_GoalsHalfTime,away_GoalsHalfTime,...,home_total_assists_cat,away_total_assists_cat,home_corners_cat,away_corners_cat,home_Goals_cat,away_Goals_cat,home_total_blocked_shots_cat,away_total_blocked_shots_cat,home_total_saved_shots_cat,away_total_saved_shots_cat
0,81,1,2015,2015-08-08 15:45:00,89,82,1,0,1,0,...,0,0,1,2,1,0,4,3,1,4
1,82,1,2015,2015-08-08 18:00:00,73,71,0,1,0,0,...,0,1,6,3,0,1,2,2,2,2
2,83,1,2015,2015-08-08 18:00:00,72,90,2,2,0,1,...,2,1,8,2,2,2,2,3,3,3
3,84,1,2015,2015-08-08 18:00:00,75,77,4,2,3,0,...,2,2,6,3,4+,2,4,2,4,3
4,85,1,2015,2015-08-08 18:00:00,79,78,1,3,0,1,...,1,3+,1,4,1,3,3,2,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12675,16131,5,2020,2021-05-23 19:00:00,168,166,1,2,1,1,...,1,1,9,5,1,2,6,1,4,3
12676,16132,5,2020,2021-05-23 19:00:00,177,176,1,2,1,1,...,1,1,5,2,1,2,3,4,2,3
12677,16133,5,2020,2021-05-23 19:00:00,163,235,2,0,1,0,...,1,0,4,3,2,0,2,0,4,2
12678,16134,5,2020,2021-05-23 19:00:00,175,181,0,1,0,1,...,0,1,9,3,0,1,5,1,5,1


,gameID,playerID,goals,ownGoals,shots,xGoals,xGoalsChain,xGoalsBuildup,assists,keyPasses,xAssists,positionOrder,yellowCard,redCard,time,subOut,subIn,leagueID
0,81,560,0,0,0,0.000000,0.000000,0.000000,0,0,0.000000,1,0,0,90,0,0,1
1,81,557,0,0,0,0.000000,0.106513,0.106513,0,1,0.106513,2,0,0,82,1,0,1
2,81,548,0,0,0,0.000000,0.127738,0.127738,0,0,0.000000,3,0,0,90,0,0,1
3,81,628,0,0,0,0.000000,0.106513,0.106513,0,0,0.000000,3,0,0,90,0,0,1
4,81,1006,0,0,0,0.000000,0.021225,0.021225,0,0,0.000000,4,0,0,90,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356508,16135,3509,0,0,1,0.028713,0.058495,0.058495,0,0,0.000000,9,0,0,90,0,0,5
356509,16135,4882,0,0,1,0.013482,0.058495,0.045374,0,2,0.058495,9,0,0,90,0,0,5
356510,16135,5786,0,0,0,0.000000,0.045374,0.045374,0,0,0.000000,15,0,0,90,0,0,5
356511,16135,8997,0,0,0,0.000000,0.000000,0.000000,0,0,0.000000,15,0,0,78,1,0,5


,gameID,leagueID,season,date,homeTeamID,awayTeamID,homeGoals,awayGoals,homeGoalsHalfTime,awayGoalsHalfTime
0,81,1,2015,2015-08-08 15:45:00,89,82,1,0,1,0
1,82,1,2015,2015-08-08 18:00:00,73,71,0,1,0,0
2,83,1,2015,2015-08-08 18:00:00,72,90,2,2,0,1
3,84,1,2015,2015-08-08 18:00:00,75,77,4,2,3,0
4,85,1,2015,2015-08-08 18:00:00,79,78,1,3,0,1
...,...,...,...,...,...,...,...,...,...,...
12675,16131,5,2020,2021-05-23 19:00:00,168,166,1,2,1,1
12676,16132,5,2020,2021-05-23 19:00:00,177,176,1,2,1,1
12677,16133,5,2020,2021-05-23 19:00:00,163,235,2,0,1,0
12678,16134,5,2020,2021-05-23 19:00:00,175,181,0,1,0,1


,gameID,shooterID,assisterID,minute,situation,lastAction,shotType,shotResult,xGoal,positionX,positionY
0,81,554,NaN,27,DirectFreekick,Standard,LeftFoot,BlockedShot,0.104347,0.794,0.421
1,81,555,631.0,27,SetPiece,Pass,RightFoot,BlockedShot,0.064342,0.860,0.627
2,81,554,629.0,35,OpenPlay,Pass,LeftFoot,BlockedShot,0.057157,0.843,0.333
3,81,554,NaN,35,OpenPlay,Tackle,LeftFoot,MissedShots,0.092141,0.848,0.533
4,81,555,654.0,40,OpenPlay,BallRecovery,RightFoot,BlockedShot,0.035742,0.812,0.707
...,...,...,...,...,...,...,...,...,...,...,...
324538,16135,6615,8651.0,19,SetPiece,Aerial,Head,MissedShots,0.039172,0.899,0.511
324539,16135,6615,8651.0,54,SetPiece,Cross,LeftFoot,Goal,0.372809,0.902,0.482
324540,16135,3464,NaN,70,OpenPlay,None,LeftFoot,MissedShots,0.009242,0.738,0.774
324541,16135,8651,4882.0,72,OpenPlay,Cross,Head,BlockedShot,0.045374,0.882,0.543


,gameID,teamID,playerID,playerName,teamName
0,81,89,560,Sergio Romero,Manchester United
1,81,89,557,Matteo Darmian,Manchester United
2,81,89,548,Daley Blind,Manchester United
3,81,89,628,Chris Smalling,Manchester United
4,81,89,1006,Luke Shaw,Manchester United
...,...,...,...,...,...
356508,16135,179,3509,Fabien Lemoine,Lorient
356509,16135,179,4882,Laurent Abergel,Lorient
356510,16135,179,5786,Yoane Wissa,Lorient
356511,16135,179,8997,Terem Igobor Moffi,Lorient


In [50]:
display(df.info())
display(df_appearances.info())
display(df_games.info())
display(df_shots.info())
display(df_player_game_team_mapping.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12680 entries, 0 to 12679
Data columns (total 63 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   gameID                        12680 non-null  int64         
 1   leagueID                      12680 non-null  int64         
 2   season                        12680 non-null  int64         
 3   date                          12680 non-null  datetime64[ns]
 4   homeTeamID                    12680 non-null  int64         
 5   awayTeamID                    12680 non-null  int64         
 6   home_Goals                    12680 non-null  int64         
 7   away_Goals                    12680 non-null  int64         
 8   home_GoalsHalfTime            12680 non-null  int64         
 9   away_GoalsHalfTime            12680 non-null  int64         
 10  home_xGoals                   12680 non-null  float64       
 11  home_shots                  

None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 356513 entries, 0 to 356512
Data columns (total 18 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   gameID         356513 non-null  int64  
 1   playerID       356513 non-null  int64  
 2   goals          356513 non-null  int64  
 3   ownGoals       356513 non-null  int64  
 4   shots          356513 non-null  int64  
 5   xGoals         356513 non-null  float64
 6   xGoalsChain    356513 non-null  float64
 7   xGoalsBuildup  356513 non-null  float64
 8   assists        356513 non-null  int64  
 9   keyPasses      356513 non-null  int64  
 10  xAssists       356513 non-null  float64
 11  positionOrder  356513 non-null  int64  
 12  yellowCard     356513 non-null  int64  
 13  redCard        356513 non-null  int64  
 14  time           356513 non-null  int64  
 15  subOut         356513 non-null  int64  
 16  subIn          356513 non-null  int64  
 17  leagueID       356513 non-nul

None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12680 entries, 0 to 12679
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   gameID             12680 non-null  int64 
 1   leagueID           12680 non-null  int64 
 2   season             12680 non-null  int64 
 3   date               12680 non-null  object
 4   homeTeamID         12680 non-null  int64 
 5   awayTeamID         12680 non-null  int64 
 6   homeGoals          12680 non-null  int64 
 7   awayGoals          12680 non-null  int64 
 8   homeGoalsHalfTime  12680 non-null  int64 
 9   awayGoalsHalfTime  12680 non-null  int64 
dtypes: int64(9), object(1)
memory usage: 990.8+ KB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 324543 entries, 0 to 324542
Data columns (total 11 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   gameID      324543 non-null  int64  
 1   shooterID   324543 non-null  int64  
 2   assisterID  240199 non-null  float64
 3   minute      324543 non-null  int64  
 4   situation   324543 non-null  object 
 5   lastAction  324543 non-null  object 
 6   shotType    324543 non-null  object 
 7   shotResult  324543 non-null  object 
 8   xGoal       324543 non-null  float64
 9   positionX   324543 non-null  float64
 10  positionY   324543 non-null  float64
dtypes: float64(4), int64(3), object(4)
memory usage: 27.2+ MB


None

<class 'pandas.core.frame.DataFrame'>
Int64Index: 356513 entries, 0 to 356512
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   gameID      356513 non-null  int64 
 1   teamID      356513 non-null  int64 
 2   playerID    356513 non-null  int64 
 3   playerName  356513 non-null  object
 4   teamName    356513 non-null  object
dtypes: int64(3), object(2)
memory usage: 16.3+ MB


None

# Datetime

In [51]:
def process_date_column(df, date_col):
    """
    Splits a datetime column into components and adds extra time features:
      - year, month, day extracted from the date.
      - time_category: categorizes the hour as 'Noon' (10-13), 'After Noon' (14-17), or 'Night' (else).
      - is_weekend: True if the day is Saturday or Sunday.
      - day_diff: difference in days between consecutive rows after sorting by the date.

    Parameters
    ----------
    df : pd.DataFrame
        DataFrame containing the date column.
    date_col : str
        Name of the date column (must be convertible to datetime).

    Returns
    -------
    pd.DataFrame
        DataFrame with new columns: 'year', 'month', 'day', 'time_category', 'is_weekend', and 'day_diff'.
    """
    df = df.copy()
    df[date_col] = pd.to_datetime(df[date_col])
    
    # Basic date components
    df['year'] = df[date_col].dt.year
    df['month'] = df[date_col].dt.month
    df['day'] = df[date_col].dt.day
    
    # Categorize time of day based on the hour
    def categorize_time(dt):
        hour = dt.hour
        if 10 <= hour < 14:
            return 'Noon'
        elif 14 <= hour < 18:
            return 'After Noon'
        else:
            return 'Night'
    df['time_category'] = df[date_col].apply(categorize_time)
    
    # Create a boolean indicator for weekend (Saturday or Sunday)
    df['is_weekend'] = df[date_col].dt.weekday >= 4
    
    # # Sort by date and compute the difference in days between consecutive games      ### Have to work on a better way to do that a code in backup_notebooks
    # df = df.sort_values(date_col).reset_index(drop=True)
    # df['day_diff'] = df[date_col].diff().dt.days.fillna(0).astype(int)
    
    return df

In [52]:
df = process_date_column(df, 'date')

In [53]:
df[['gameID','date','year','month','day','time_category','is_weekend']]

,gameID,date,year,month,day,time_category,is_weekend
0,81,2015-08-08 15:45:00,2015,8,8,After Noon,True
1,82,2015-08-08 18:00:00,2015,8,8,Night,True
2,83,2015-08-08 18:00:00,2015,8,8,Night,True
3,84,2015-08-08 18:00:00,2015,8,8,Night,True
4,85,2015-08-08 18:00:00,2015,8,8,Night,True
...,...,...,...,...,...,...,...
12675,16131,2021-05-23 19:00:00,2021,5,23,Night,True
12676,16132,2021-05-23 19:00:00,2021,5,23,Night,True
12677,16133,2021-05-23 19:00:00,2021,5,23,Night,True
12678,16134,2021-05-23 19:00:00,2021,5,23,Night,True


it seems to be correct :)

In [54]:
for ft in ['year','month','day','time_category','is_weekend']:
    display(df[ft].value_counts(dropna=False))

2017    1877
2015    1846
2019    1823
2016    1812
2018    1808
2020    1570
2021    1093
2014     851
Name: year, dtype: int64

12    1527
4     1457
2     1440
9     1314
1     1287
10    1207
11    1183
3     1077
5     1046
8      779
7      211
6      152
Name: month, dtype: int64

20    567
21    532
22    481
23    471
19    463
1     453
3     451
18    438
26    436
2     428
17    427
16    419
24    414
28    414
4     408
14    400
6     397
29    392
25    391
13    390
30    383
7     383
15    380
10    374
9     367
5     367
8     366
27    359
12    348
11    318
31    263
Name: day, dtype: int64

Night         6169
After Noon    5278
Noon          1233
Name: time_category, dtype: int64

True     10632
False     2048
Name: is_weekend, dtype: int64

# Historical Performance & Goal Metrics